### 1. 데이터와 라이브러리 로드하기
1.1 라이브러리 로드   
1.2 데이터 로드

In [1]:
import pandas as pd

In [192]:
market_csv = pd.read_csv('../Data/infra_data/서울시 대규모점포 인허가 정보.csv', encoding='cp949')
market_csv.keys()

Index(['개방자치단체코드', '관리번호', '인허가일자', '인허가취소일자', '영업상태코드', '영업상태명', '상세영업상태코드',
       '상세영업상태명', '폐업일자', '휴업시작일자', '휴업종료일자', '재개업일자', '전화번호', '소재지면적',
       '소재지우편번호', '지번주소', '도로명주소', '도로명우편번호', '사업장명', '최종수정일자', '데이터갱신구분',
       '데이터갱신일자', '업태구분명', '좌표정보(X)', '좌표정보(Y)', '점포구분명'],
      dtype='object')

### 2. 중요한 column 만 남기기
1.1 지번 주소중 결측치 채워넣기   
1.2 주소에서 동만 뽑아서 넣기

In [179]:
market_df = market_csv[['사업장명', '지번주소', '인허가일자', '폐업일자']]
market_df[pd.isnull(market_df['지번주소'])]

,사업장명,지번주소,인허가일자,폐업일자
14,(주)이마트에브리데이 암사동점,NaN,2011-11-28,2023-03-24
15,동화상가,NaN,19680730,NaN
47,누죤,NaN,19960705,NaN
78,삼익패션타운,NaN,19851128,NaN
79,중부시장,NaN,19590224,NaN
...,...,...,...,...
960,(주)이랜드킴스클럽 강남점,NaN,19950601,NaN
962,롯데쇼핑(주)롯데슈퍼신당점,NaN,20111007,NaN
963,롯데프리미엄푸드마켓 공덕점,NaN,20090529,NaN
964,롯데슈퍼 대흥점,NaN,20080812,NaN


### 1.1 지번 주소중 결측치 채워넣기
- KAKAO API를 이용해서 사업장명으로부터 지번주소를 얻는 함수를 만들기
- 함수를 이용해서 지번주소 map

In [92]:
import requests
import json
import pprint

def get_address(market_name):
    url = 'https://dapi.kakao.com/v2/local/search/keyword.json'

    query_string = {'query': market_name}
    header = {'Authorization': 'KakaoAK 90ef8b6765eda8828495848108967f9e'}

    response = requests.get(url, headers=header, params=query_string)
    tokens = response.json()

    if len(tokens['documents']) > 0:

        return tokens['documents'][0]['address_name']
    
    else:

        return 'Unknown Unknown Unknwon'


In [180]:
market_df.loc[:,'지번주소'] = market_df['지번주소'].fillna(market_df['사업장명'].map(get_address))

In [181]:
market_df[market_df['지번주소'].str.contains('Unknown')]

,사업장명,지번주소,인허가일자,폐업일자
85,남정종합상가,Unknown Unknown Unknwon,20020722,NaN
100,DDP FASHION MALL,Unknown Unknown Unknwon,20170323,NaN
111,지에스수퍼마켓 중구만리점,Unknown Unknown Unknwon,20171031,NaN
146,(주)지에스리테일 남산타운점,Unknown Unknown Unknwon,20140110,NaN
148,굳앤굳 디자이너월드,Unknown Unknown Unknwon,19761102,NaN
167,(주)심풍유통,Unknown Unknown Unknwon,19851109,20150827
172,서소문화훼종합물류센터,Unknown Unknown Unknwon,20020201,20150827
196,롯데마켓999 용산파크자이가맹점,Unknown Unknown Unknwon,20140805,20180612
370,홈플러스(주)익스프레스미아점,Unknown Unknown Unknwon,20120322,NaN
387,롯데쇼핑(주)롯데슈퍼미아점,Unknown Unknown Unknwon,20120320,20190227


### Kakao map으로 못찾은 장소 제외하기
1. 주소에서 동만 추출하기
2. Unknown으로 되어있거나 '동'을 미포함한 장소는 제외하기

In [182]:
market_df.loc[:, '지번주소'] = market_df['지번주소'].map(lambda x: x.split()[2] if len(x.split()) > 2 else x)
market_df = market_df[market_df['지번주소'].str.contains('동')]
market_df.reset_index(inplace=True, drop=True)
market_df


,사업장명,지번주소,인허가일자,폐업일자
0,마리오아울렛 1관,가산동,2013-09-06,NaN
1,마리오아울렛 3관,가산동,2012-09-13,NaN
2,롯데슈퍼 목동2점,목5동,2012-04-05,NaN
3,영등포유통상가,당산동2가,1999-05-31,NaN
4,롯데슈퍼 방배2점,방배1동,2012-04-24,NaN
...,...,...,...,...
973,롯데마이슈퍼 장위점,장위동,20120521,NaN
974,롯데마이슈퍼 정릉점,정릉3동,20120521,NaN
975,롯데쇼핑(주) 롯데슈퍼 역촌점,역촌동,20120320,NaN
976,롯데백화점 건대스타시티점,자양동,20080822,NaN


### 3. 동별로 묶어서 데이터 추가하기
3.1 동별로 개수 구하기   
3.2 구한 개수 기존 데이터랑 묶기

In [195]:
market_group = market_df.groupby('지번주소')
market_group = market_group['사업장명'].count()
market_group

지번주소
가락2동     1
가락동      6
가산동      7
가양동      5
갈현1동     1
        ..
회기동      4
회현동1가    2
휘경동      1
흑석동      1
흥인동      1
Name: 사업장명, Length: 314, dtype: int64

In [204]:
seoul_df = pd.read_csv('./서울_한글.csv')
df_inner_join = pd.merge(seoul_df, market_group, left_on='동', right_on='지번주소', how='inner')


In [210]:
df_inner_join = df_inner_join.rename(columns={'사업장명':'마트개수'})
df_inner_join

,시/군/구,동,전월세구분,전용면적(㎡),보증금(만원),월세(만원),층,구분,한국은행기준금리,종가,년도,건물나이,시군구_Encoder,동_Encoder,마트개수
0,강남구,개포동,0,77.75,33000,0,2.0,0,2.50,1941.15,2014.0,26.0,0,9,3
1,강남구,개포동,0,79.97,34000,0,2.0,0,2.50,1941.15,2014.0,27.0,0,9,3
2,강남구,개포동,0,79.97,28000,0,1.0,0,2.50,1941.15,2014.0,27.0,0,9,3
3,강남구,개포동,0,79.97,35000,0,2.0,0,2.50,1941.15,2014.0,27.0,0,9,3
4,강남구,개포동,1,104.43,40000,40,8.0,0,2.50,1941.15,2014.0,30.0,0,9,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3114885,중구,산림동,1,91.98,200,20,8.0,0,1.25,2402.71,2017.0,45.0,23,155,1
3114886,중구,산림동,1,91.98,200,25,8.0,0,1.25,2391.79,2017.0,45.0,23,155,1
3114887,중구,산림동,0,66.78,17000,0,12.0,0,1.50,2029.47,2015.0,43.0,23,155,1
3114888,중구,산림동,1,68.10,2500,50,9.0,0,1.50,2029.47,2015.0,43.0,23,155,1
